In [2]:
from sentence_transformers import SentenceTransformer
from transformers import GPT2Model, GPT2Tokenizer
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tqdm.notebook import tqdm_notebook
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
from tqdm import tqdm
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import T5Model, T5Tokenizer

from sentence_transformers import SentenceTransformer
from transformers import GPT2Model, GPT2Tokenizer
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tqdm.notebook import tqdm_notebook
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
from tqdm import tqdm
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from tqdm.notebook import tqdm
import pandas as pd
import random
import torch
from transformers import DebertaTokenizer, DebertaModel
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim

In [3]:
# Set device for model training and inference
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

train_data_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_sp.csv'
train_data_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_wp.csv'

# Initialize DeBERTa tokenizer and model
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
embedding_model = DebertaModel.from_pretrained('microsoft/deberta-base').to(device)
embedding_model.eval()  # Set the model to evaluation mode

def make_dataset(data):
    x_train = []
    y_train = []
    with torch.no_grad():
        for idx, row in data.iterrows():
            question = row['question']
            answer = row['answer']
            distractor1 = row['distractor1']
            distractor2 = row['distractor2']

            # Tokenize and encode each pair
            q_a_input = tokenizer(question, answer, return_tensors="pt", padding=True, truncation=True).to(device)
            q_d1_input = tokenizer(question, distractor1, return_tensors="pt", padding=True, truncation=True).to(device)
            q_d2_input = tokenizer(question, distractor2, return_tensors="pt", padding=True, truncation=True).to(device)
            
            # Obtain pooled output from DeBERTa
            q_a_emb = embedding_model(**q_a_input).last_hidden_state[:, 0]
            q_d1_emb = embedding_model(**q_d1_input).last_hidden_state[:, 0]
            q_d2_emb = embedding_model(**q_d2_input).last_hidden_state[:, 0]

            # Concatenate embeddings and append labels
            x_train.append(torch.cat([q_a_emb, q_d1_emb], dim=1))
            y_train.append(torch.tensor([0], dtype=torch.float32))

            x_train.append(torch.cat([q_a_emb, q_d2_emb], dim=1))
            y_train.append(torch.tensor([0], dtype=torch.float32))

            x_train.append(torch.cat([q_a_emb, q_a_emb], dim=1))  # Correct combination for a positive example
            y_train.append(torch.tensor([1], dtype=torch.float32))

    x_train = torch.stack(x_train).squeeze(1)
    y_train = torch.stack(y_train).view(-1, 1)
    return x_train, y_train

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(1536, 768)
        self.dropout1 = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(768, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.linear3 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.sigmoid(x)
        return x

def train(model, loader, epochs):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        total_loss = 0
        for x_batch, y_batch in loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(x_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {total_loss / len(loader)}')

# Example usage
# Example usage
csv_data_sp = pd.read_csv(train_data_dir_sp)
csv_data_wp = pd.read_csv(train_data_dir_wp)
x_train_sp, y_train_sp = make_dataset(csv_data_sp)
x_train_wp, y_train_wp = make_dataset(csv_data_wp)

# Create a DataLoader
train_loader_sp = DataLoader(TensorDataset(x_train_sp, y_train_sp), batch_size=32, shuffle=True)
train_loader_wp = DataLoader(TensorDataset(x_train_wp, y_train_wp), batch_size=32, shuffle=True)

# Initialize and train the model
model = Model().to(device)
train(model, train_loader_sp, 10)
train(model, train_loader_wp, 10)
torch.save(model, '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/Binary-classification-version/deberta.pth')


Epoch 1, Loss: 0.6485530187686285
Epoch 2, Loss: 0.6472088427593311
Epoch 3, Loss: 0.6378699770818154
Epoch 4, Loss: 0.648571077734232
Epoch 5, Loss: 0.6419491643706957
Epoch 6, Loss: 0.6445453949272633
Epoch 7, Loss: 0.6477380730211735
Epoch 8, Loss: 0.644877896954616
Epoch 9, Loss: 0.6402731699248155
Epoch 10, Loss: 0.6441772989928722
Epoch 1, Loss: 0.6478976121074275
Epoch 2, Loss: 0.650036118532482
Epoch 3, Loss: 0.6361600835072366
Epoch 4, Loss: 0.6475583456064525
Epoch 5, Loss: 0.6429201835080197
Epoch 6, Loss: 0.6432400157577113
Epoch 7, Loss: 0.6493970673335226
Epoch 8, Loss: 0.6507728680184013
Epoch 9, Loss: 0.649108067939156
Epoch 10, Loss: 0.6377576871922142


In [3]:
import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset


def inference(model, question, choices):
    model.eval()
    logits = []
    with torch.no_grad():  # Ensure no gradients are calculated during inference
        inputs_question = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs_question = embedding_model(**inputs_question)
        question_embedding = outputs_question.last_hidden_state[:, 0]

        for choice in choices:
            inputs_choice = tokenizer(choice, return_tensors="pt", padding=True, truncation=True).to(device)
            outputs_choice = embedding_model(**inputs_choice)
            choice_embedding = outputs_choice.last_hidden_state[:, 0]

            # Concatenate along dimension 1 to form a single vector [1536]
            combined_input = torch.cat((question_embedding, choice_embedding), dim=1).unsqueeze(0)

            logit = model(combined_input)
            logits.append(logit)

    probabilities = torch.cat(logits).sigmoid()
    predicted_index = torch.argmax(probabilities).item()
    return choices[predicted_index]



def predict(model, df):
    for i, row in df.iterrows():
        question = row['question']
        choices = eval(row['choice_list']) if isinstance(row['choice_list'], str) else row['choice_list']
        prediction = inference(model, question, choices)
        predicted_id = choices.index(prediction) if prediction in choices else -1
        df.loc[i, 'pred_id'] = int(predicted_id)
    return df

def write_answer_id(df, path):
    with open(path, 'w', encoding='utf-8') as f:
        for i, row in df.iterrows():
            f.write(f"{int(row['pred_id'])}\n")

test_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_sp.csv'
test_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_wp.csv'
test_df_sp = pd.read_csv(test_dir_sp)
test_df_wp = pd.read_csv(test_dir_wp)
test_df_sp = predict(model, test_df_sp)  # model should be your trained Bert-based model
test_df_wp = predict(model, test_df_wp) 

test_df_sp.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_deberta_sp.csv")
test_df_wp.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_deberta_wp.csv")


In [4]:
import pandas as pd

# Load your DataFrame
df_sp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_deberta_sp.csv')
df_wp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_deberta_wp.csv')

# Ensure 'id' is treated as string and replace NaN with an empty string if necessary
df_sp['id'] = df_sp['id'].astype(str)
df_wp['id'] = df_wp['id'].astype(str)

# Define function to calculate accuracy
def calculate_accuracy(df, true_label_col, pred_label_col):
    correct_predictions = df[df[true_label_col] == df[pred_label_col]]
    accuracy = len(correct_predictions) / len(df)
    return accuracy

# Calculate accuracies for each subset using the updated string column
S_ori = calculate_accuracy(df_sp[~df_sp['id'].str.contains("_")], 'label', 'pred_id')
S_sem = calculate_accuracy(df_sp[df_sp['id'].str.contains("_SR")], 'label', 'pred_id')
S_con = calculate_accuracy(df_sp[df_sp['id'].str.contains("_CR")], 'label', 'pred_id')
S_ori_sem = calculate_accuracy(df_sp[(df_sp['id'].str.contains("_SR") | ~df_sp['id'].str.contains("_"))], 'label', 'pred_id')
S_ori_sem_con = calculate_accuracy(df_sp, 'label', 'pred_id')  # For all data
S_overall = S_ori_sem_con  # Overall is the same as considering all data

W_ori = calculate_accuracy(df_wp[~df_wp['id'].str.contains("_")], 'label', 'pred_id')
W_sem = calculate_accuracy(df_wp[df_wp['id'].str.contains("_SR")], 'label', 'pred_id')
W_con = calculate_accuracy(df_wp[df_wp['id'].str.contains("_CR")], 'label', 'pred_id')
W_ori_sem = calculate_accuracy(df_wp[(df_wp['id'].str.contains("_SR") | ~df_wp['id'].str.contains("_"))], 'label', 'pred_id')
W_ori_sem_con = calculate_accuracy(df_wp, 'label', 'pred_id')  # For all data
W_overall = S_ori_sem_con  # Overall is the same as considering all data


# Print the calculated scores
print(f'S_ori: {S_ori}')
print(f'S_sem: {S_sem}')
print(f'S_con: {S_con}')
print(f'S_ori_sem: {S_ori_sem}')
print(f'S_ori_sem_con: {S_ori_sem_con}')
print(f'S_overall: {S_overall}')

# Print the calculated scores
print(f'W_ori: {W_ori}')
print(f'W_sem: {W_sem}')
print(f'W_con: {W_con}')
print(f'W_ori_sem: {W_ori_sem}')
print(f'W_ori_sem_con: {W_ori_sem_con}')
print(f'W_overall: {W_overall}')


S_ori: 0.35502958579881655
S_sem: 0.34911242603550297
S_con: 0.38461538461538464
S_ori_sem: 0.3520710059171598
S_ori_sem_con: 0.3629191321499014
S_overall: 0.3629191321499014
W_ori: 0.12878787878787878
W_sem: 0.13636363636363635
W_con: 0.10606060606060606
W_ori_sem: 0.13257575757575757
W_ori_sem_con: 0.12373737373737374
W_overall: 0.3629191321499014
